In [1]:
!pip install konlpy

In [2]:
!pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 13.2 MB/s eta 0:00:00


In [3]:
!pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   -- ------------------------------------- 1.6/24.0 MB 8.4 MB/s eta 0:00:03
   ------ --------------------------------- 4.2/24.0 MB 10.1 MB/s eta 0:00:02
   ---------- ----------------------------- 6.0/24.0 MB 9.5 MB/s eta 0:00:02
   -------------- ------------------------- 8.7/24.0 MB 10.3 MB/s eta 0:00:02
   ------------------ --------------------- 11.3/24.0 MB 10.7 MB/s eta 0:00:02
   ----------------------- ---------------- 13.9/24.0 MB 11.0 MB/s eta 0:00:01
   --------------------------- ------------ 16.5/24.0 MB 11.3 MB/s eta 0:00:01
   ------------------------------- -------- 18.9/24.0 MB 11.3 MB/s eta 0:00:01
   --------------------------------- ------ 20.2/24.0 MB 10.6 MB/s eta 0:00:01
   ------------------------------------- -- 22.5/24.0 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 24.0/24.0 MB 10.6 MB/s eta 0:00:00


In [49]:
import re
from konlpy.tag import Okt
from collections import Counter

In [50]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어." # 토큰화와 전처리가 전혀 되어 있지 않은 텍트트
text

'임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'

### 정규 표현식을 반영하여 전처리

In [51]:
reg = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]") # 자음, 모음, 한글 글자, 공백 제외 문자를 남기고 나머지 제거
text = reg.sub('', text)
print(text)

임금님 귀는 당나귀 귀 임금님 귀는 당나귀 귀 실컷 소리치고 나니 속이 확 뚫려 살 것 같았어


### 토큰화 KoNLPy Okt

In [7]:
okt=Okt()
tokens = okt.morphs(text)
print(tokens)

['임금님', '귀', '는', '당나귀', '귀', '임금님', '귀', '는', '당나귀', '귀', '실컷', '소리', '치고', '나니', '속이', '확', '뚫려', '살', '것', '같았어']


### 단어장 만들기

In [8]:
vocab = Counter(tokens)
print(vocab)

Counter({'귀': 4, '임금님': 2, '는': 2, '당나귀': 2, '실컷': 1, '소리': 1, '치고': 1, '나니': 1, '속이': 1, '확': 1, '뚫려': 1, '살': 1, '것': 1, '같았어': 1})


In [9]:
vocab['임금님']

2

In [10]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('귀', 4), ('임금님', 2), ('는', 2), ('당나귀', 2), ('실컷', 1)]


In [11]:
word2idx={word[0] : index+1 for index, word in enumerate(vocab)}
print(word2idx)

{'귀': 1, '임금님': 2, '는': 3, '당나귀': 4, '실컷': 5}


### 원-핫 벡터 만들기

In [29]:
def one_hot_encoding(word, word2index):
       one_hot_vector = [0]*(len(word2index))
       index = word2index[word]
       one_hot_vector[index-1] = 1
       return one_hot_vector

In [13]:
one_hot_encoding("임금님", word2idx)

[0, 1, 0, 0, 0]

### 케라스를 통한 원-핫 인코딩 one-hot encoding

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [15]:
text = [['강아지', '고양이', '강아지'],['애교', '고양이'], ['컴퓨터', '노트북']]
text

[['강아지', '고양이', '강아지'], ['애교', '고양이'], ['컴퓨터', '노트북']]

In [16]:
t = Tokenizer()
t.fit_on_texts(text)
print(t.word_index) # 각 단어에 대한 인코딩 결과 출력.

{'강아지': 1, '고양이': 2, '애교': 3, '컴퓨터': 4, '노트북': 5}


In [33]:
vocab_size = len(t.word_index) + 1

In [18]:
sub_text = ['강아지', '고양이', '강아지', '컴퓨터']
encoded = t.texts_to_sequences([sub_text])
print(encoded)

[[1, 2, 1, 4]]


### 원-핫 벡터의 시퀀스로 변환

In [19]:
one_hot = to_categorical(encoded, num_classes = vocab_size)
print(one_hot)

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]]]


## 영어 Word2Vec 실습과 OOV

In [20]:
import nltk
nltk.download('abc')

## 이부분 punkt 대신 punkt_tab을 이용
nltk.download('punkt_tab')

[nltk_data] Downloading package abc to
[nltk_data]     C:\Users\juwonhee\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\abc.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\juwonhee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [34]:
from nltk.corpus import abc
corpus = abc.sents()

In [22]:
print(corpus[:3])

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ['In', 'one', 'of', 'the', 'letters', 'Mr', 'Howard', 'asks', 'AWB', 'managing', 'director', 'Andrew', 'Lindberg', 'to', 'remain', 'in', 'close', 'contact', 'with', 'the', 'Government', 'on', 'Iraq', 'wheat', 'sales', '.']]


In [23]:
print('코퍼스의 크기 :',len(corpus))

코퍼스의 크기 : 29059


### Word2Vec 훈련
- vector size = 학습 후 임베딩 벡터의 차원
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도수 제한 (빈도가 적은 단어들은 학습하지 않아요.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBoW, 1은 Skip-gram.

In [24]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
print("모델 학습 완료!")

모델 학습 완료!


- 모델이 어떤 학습을 마친걸까?  
단어를 벡터(숫자)로 표현하는 방법을 학습했다는 의미  
sg=0 이기 때문에 CBow 방법을 사용

Word2Vec는 입력한 단어에 대해서 가장 코사인 유사도가 높은 단어들을 출력하는
-> model.wv.most_similar

In [25]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.9233420491218567), ('skull', 0.9110078811645508), ('Bang', 0.9056522250175476), ('asteroid', 0.9051969051361084), ('third', 0.9020065665245056), ('baby', 0.8994149565696716), ('dog', 0.8985928893089294), ('bought', 0.8975393772125244), ('rally', 0.8912544846534729), ('disc', 0.8888999223709106)]


### 모델 저장 및 로드

In [35]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('./w2v')
loaded_model = KeyedVectors.load_word2vec_format("./w2v")
print("모델  load 완료!")

모델  load 완료!


In [52]:
model_result = loaded_model.most_similar("man")
print(model_result) # (유사단어, 유사도 점수)

[('woman', 0.9233420491218567), ('skull', 0.9110078811645508), ('Bang', 0.9056522250175476), ('asteroid', 0.9051969051361084), ('third', 0.9020065665245056), ('baby', 0.8994149565696716), ('dog', 0.8985928893089294), ('bought', 0.8975393772125244), ('rally', 0.8912544846534729), ('disc', 0.8888999223709106)]


## Word2Vec의 OOV 문제 (Out of Vocabuary)
Bag of Words 학습 과정에서 언급되었던 문제인, 미등록 단어 문제와  
오타 단어는 에러를 발생시킨다.  


In [36]:
# 에러나는 코드가 맞음
loaded_model.most_similar('overacting')
loaded_model.most_similar('memorry')

KeyError: "Key 'overacting' not present in vocabulary"

## 임베딩 벡터의 시각화

앞서 Word2Vec 모델이 단어들을 100차원의 숫자 벡터로 변환  
그 숫자 벡터들이 실제로 어떤 의미를 가지고, 어떻게 배치되어 있는지 시각화하기 위해  
임베딩 프로젝터를 사용한다.

In [37]:
# word2vec 모델 메타정보 및 텐서 내보내기
!python -m gensim.scripts.word2vec2tensor --input ./w2v --output ./w2v

2025-06-17 12:50:09,426 - word2vec2tensor - INFO - running C:\Users\juwonhee\AppData\Local\Programs\Python\Python38\lib\site-packages\gensim\scripts\word2vec2tensor.py --input ./w2v --output ./w2v
2025-06-17 12:50:09,426 - keyedvectors - INFO - loading projection weights from ./w2v
2025-06-17 12:50:09,983 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (10363, 100) matrix of type float32 from ./w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2025-06-17T12:50:09.983717', 'gensim': '4.3.3', 'python': '3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'load_word2vec_format'}
2025-06-17 12:50:10,628 - word2vec2tensor - INFO - 2D tensor file saved to ./w2v_tensor.tsv
2025-06-17 12:50:10,628 - word2vec2tensor - INFO - Tensor metadata file saved to ./w2v_metadata.tsv
2025-06-17 12:50:10,628 - word2vec2tensor - INFO - finished running word2vec2tensor.py


## FastText
Word2Vec은 더 이상 깨질 수 없는 단위로 단어를 구분하는 반면,  
FastText는 단어 내부의 내부 단어를 학습한다는 아이디어를 가지고 있다.  
네거티브 샘플링을 사용하여 학습한다.  
다만, Word2Vec과 다른 점은 학습 과정에서 중심 단어에 속한 문자 단위 단어 벡터들을 모두 업데이트한다.
- OOV와 오타에 강건하다

In [38]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)
print("FastText 학습 완료!")

FastText 학습 완료!


In [39]:
fasttext_model.wv.most_similar('overacting')

[('fluctuating', 0.9377561211585999),
 ('resolving', 0.9375224709510803),
 ('emptying', 0.9363172650337219),
 ('extracting', 0.9336783289909363),
 ('malting', 0.9330613613128662),
 ('mounting', 0.9304724335670471),
 ('shooting', 0.9295411109924316),
 ('lifting', 0.9291247725486755),
 ('declining', 0.928656280040741),
 ('overwhelming', 0.9271591901779175)]

In [40]:
fasttext_model.wv.most_similar('memoryy')

[('memory', 0.9466140270233154),
 ('musical', 0.8680832386016846),
 ('mechanisms', 0.8637781739234924),
 ('mechanism', 0.8612226247787476),
 ('basic', 0.8582766056060791),
 ('mechanical', 0.8552515506744385),
 ('imagine', 0.8516234755516052),
 ('technical', 0.8439446091651917),
 ('intelligence', 0.839522123336792),
 ('visual', 0.8367835283279419)]

memoryy 와 가장 유사한 단어로 memory가 출력된다.

## GloVe
- 카운트 기반(DTM)
- 예측 기반(Word2Vec와 같은 방법)

In [46]:
import gensim.downloader as api
glove_model = api.load("glove-wiki-gigaword-50")  # glove vectors 다운로드
glove_model.most_similar("dog")  # 'dog'과 비슷한 단어 찾기

[('cat', 0.9218006134033203),
 ('dogs', 0.8513158559799194),
 ('horse', 0.7907583117485046),
 ('puppy', 0.7754921317100525),
 ('pet', 0.7724707722663879),
 ('rabbit', 0.7720813751220703),
 ('pig', 0.7490062117576599),
 ('snake', 0.7399188280105591),
 ('baby', 0.7395570278167725),
 ('bite', 0.7387937307357788)]

In [47]:
glove_model.most_similar('overacting')

[('impudence', 0.7842012643814087),
 ('puerile', 0.7816032767295837),
 ('winningly', 0.7644237279891968),
 ('grossness', 0.7576098442077637),
 ('deconstructions', 0.748936653137207),
 ('over-the-top', 0.7460805177688599),
 ('buffoonery', 0.746045708656311),
 ('impetuosity', 0.7415392994880676),
 ('sophomoric', 0.736961841583252),
 ('zaniness', 0.7353197336196899)]

In [48]:
glove_model.most_similar('memoryy')

KeyError: "Key 'memoryy' not present in vocabulary"

GloVe는 Word2Vec와 같이 OOV 문제를 가지고 있어서 오타를 인식하지 못한다.  